In [19]:
from sklearn.datasets import load_iris
import numpy as np

x = load_iris().data
# x = np.c_[x, np.ones(x.shape[0])]
y = load_iris().target

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(112, 4)

In [38]:
def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    theta = np.random.rand(X.shape[1], 1)
    print(theta)
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            r = np.random.randint(0, m-1)
            xi, yi = X[r:r+1], y[r:r+1]
            diff = xi.T @ ((xi @ theta) - yi)
            eta = learning_schedule(epoch * m + n)
            theta = theta - (diff * eta)
    return theta

theta = SGD(x_train, y_train, 50, 0.1)

print(theta)

preds = x_test @ theta

[[ 0.42142425]
 [ 0.40800227]
 [ 0.60545502]
 [ 0.91696069]]
[[-0.0651501 ]
 [-0.03257456]
 [ 0.19174441]
 [ 0.66645465]]


In [39]:
preds = np.rint(preds)

In [40]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

0.94736842105263153

In [23]:
def scaling(X):
    l = []
    for k in range(X.shape[1]):
        x = X[:,k]
        def f(e):
            tmp0 = sum(x) / len(x)
            tmp1 = max(x) - min(x)
            return (e - tmp0) / tmp1
        m =  list(map(float, (list(map(f, x)))))
        l.append(m)
    return np.matrix(l).transpose()

In [59]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), scaling(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            gradients = ((X @ theta.T) - y.T).T @ X[:,c]
            theta[:,c] = theta[:,c] - eta * gradients
    return theta.T

theta_BGD = BGD(x_train, y_train, 1000, 0.0001)

print(theta_BGD)
print(theta)

[[ 0.91964197]
 [ 0.94754793]
 [-0.09346289]
 [ 0.78788087]
 [ 1.04140224]]
[[-0.0651501 ]
 [-0.03257456]
 [ 0.19174441]
 [ 0.66645465]]


In [55]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), scaling(X)] @ theta

In [56]:
preds = predict(x_test, theta_BGD)

preds = np.rint(preds)
accuracy_score(y_test, preds)

0.76315789473684215

In [35]:
# theta_NORM = (np.linalg.inv(x_train.T @ x_train) @ x_train.T @ y_train).reshape((5, 1))